In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize


def tokenize_split(text):
    """simple tokeniser"""
    return text.split()


# Functions for chunking transcripts on either words or sentences

def word_chunk_transcript(transcripts, chunk_size=500):
    """ transcripts: must have two columns - episode_id and tokenised_transcript
        chunk_size: number of tokens in chunk 
    """
    transcripts["tokenised_transcript"] = transcripts.transcript.apply(lambda x: tokenize_split(x))

    episode_ls = []
    transcript_ls = []
    words_enum_ls = [] 
    for index, row in transcripts.iterrows():
        for i in range(0, len(row["tokenised_transcript"]), chunk_size):
            episode_ls.append(row["episode_id"])
            transcript_ls.append(" ".join(row["tokenised_transcript"][i:i+chunk_size]))
            words_enum_ls.append(f"{i} - {i+chunk_size}")
    word_chunked_df = pd.DataFrame(data = {'episode_id': episode_ls, 'transcript_subset':transcript_ls, 'words_enumerated':words_enum_ls})
    return word_chunked_df


def sentence_chunk_transcript(transcripts, chunk_size=20):
    """ transcripts: must have two columns - episode_id and transcript
        chunk_size: number of sentences in chunk 
    """
    transcripts["sentence_token"] = transcripts.transcript.apply(lambda x: sent_tokenize(x, language='english'))
    episode_ls = []
    transcript_ls = []
    sent_enum_ls = []
    for index, row in transcripts.iterrows():
        for i in range(0, len(row["sentence_token"]), chunk_size):
            episode_ls.append(row["episode_id"])
            transcript_ls.append(" ".join(row["sentence_token"][i:i+chunk_size]))
            sent_enum_ls.append(f"{i} - {i+chunk_size}")
    sentence_chunked_df = pd.DataFrame(data = {'episode_id': episode_ls, 'transcript_subset':transcript_ls, 'sentence_enumerated':sent_enum_ls})
    return sentence_chunked_df


### Make chunked dataset

In [2]:
transcripts_sample = pd.read_csv("transcripts_sample.csv.gz", compression="gzip")
print(transcripts_sample.shape)
# transcripts_sample.head(5)

(15000, 16)


In [6]:
# Extract columns for conversion
cols_subset = transcripts_sample.loc[: ,["episode_id", "transcript"]]

# Make dataset with chunked documents on 25 sentences per chunk
sentences_25 = sentence_chunk_transcript(cols_subset, chunk_size=25)
sentences_40 = sentence_chunk_transcript(cols_subset, chunk_size=40)
print(sentences_25.shape)
print(sentences_40.shape)

# sentences_25.head(10)

(215064, 3)
(137258, 3)


In [7]:
# Check mean word count of chunks
sentences_25["word_count"] = sentences_25.transcript_subset.apply(lambda x: len(x.split(" ")))
sentences_25.word_count.mean()

400.48517650559836

In [8]:
# Check mean word count of chunks
sentences_40["word_count"] = sentences_40.transcript_subset.apply(lambda x: len(x.split(" ")))
sentences_40.word_count.mean()

627.5039997668624

In [9]:
# Make new dataset
sentences_25.to_csv("sentences_chunkssize_25.csv.gz", compression="gzip")
len(sentences_25[sentences_25.word_count == 1])

74

In [10]:
# Make new dataset
sentences_40.to_csv("sentences_chunkssize_40.csv.gz", compression="gzip")
len(sentences_40[sentences_40.word_count == 1])

48